### <center> Testes de Implementação: **Ollama e Langchain** em Agentes Inteligentes <center>


Importações

In [1]:
from pydantic import BaseModel, Field
from langchain.tools import BaseTool
from typing import Optional, Type
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain.tools.render import render_text_description_and_args
import re
import ollama
import chromadb
from langchain.prompts import PromptTemplate
from langchain_ollama import ChatOllama

Extratação do Documento

In [2]:
faq = ""

with open('faq.txt', 'r', encoding='utf-8') as file:
        faq = file.read()

documents = re.split(r'(?=Q: )', faq)

for i in reversed(documents):
    if i == "":
        documents.remove(i)

print(documents)

['[Aplicativo]\n', 'Q: "Como faço para deixar as atualizações de forma automática?"\nA: "*Anroid*\nO SEMPRE JBS foi desenvolvido para trazer maior agilidade nas informações e praticidade no seu dia a dia. Por isso, quando houver atualizações, você não precisa se preocupar em entrar no sistema e buscar essa nova versão. O aplicativo já vai te avisar que tem uma nova versão e te levar para a google store. É fácil e rápido. Não se preocupe 😊\n\n*IOS*\nCaso você queira que seu celular atualize sozinho para as próximas versões, basta:\n    1.      Clicar em “Ajustes”;\n    2.      Clicar em “Apple store”;\n    3.      Selecionar a opção de “Atualização automática”."\n\n', 'Q: "Como faço para deixar de forma automática a atualização do Sempre JBS em meu aparelho Android?"\nA: "O SEMPRE JBS foi desenvolvido para trazer maior agilidade nas informações e praticidade no seu dia a dia. Por isso, quando houver atualizações, você não precisa se preocupar em entrar no sistema e buscar essa nova vers

Conexão com o ChromaDB

In [3]:
client = chromadb.Client()
collection = client.create_collection(name="docs")

# Guardar os documentos em um vetor para embedding 
for i, d in enumerate(documents):
    response = ollama.embed(model="mxbai-embed-large", input=d)
    embeddings = response["embeddings"]
    collection.add(
        ids=[str(i)],
        embeddings=embeddings,
        documents=[d]
    )

Configuração de Tools

In [4]:
class EmbeddingInput(BaseModel):
    question: str = Field(..., description="A pergunta do usuário que requer informações técnicas ou dados.")

class EmbeddingTool(BaseTool):
    name: str = "Embedding"
    description: str = (
        "Útil quando o usuário precisa de informações técnicas ou qualquer pergunta que exija dados para fornecer uma resposta."
    )
    args_schema: Type[BaseModel] = EmbeddingInput

    def _get_embedding(self, text: str):
        """Gera embedding para o texto fornecido."""
        try:
            response = ollama.embed(model="mxbai-embed-large", input=text)
            return response["embeddings"][0]
        except Exception as e:
            raise ValueError(f"Erro ao gerar embedding: {str(e)}")

    def _query_collection(self, embedding, num_results=1):
        """Consulta o ChromaDB usando o embedding fornecido."""
        try:
            results = collection.query(
                query_embeddings=[embedding],
                n_results=num_results
            )
            return results
        except Exception as e:
            raise ValueError(f"Erro ao consultar o ChromaDB: {str(e)}")

    def _run(
        self,
        question: Optional[str] = None,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Obtém informações relevantes para a pergunta."""
        if not question:
            return "Nenhuma pergunta foi fornecida."

        try:
            embedding = self._get_embedding(question)

            results = self._query_collection(embedding, num_results=1)

            if results and results["documents"]:
                return (
                    f"Use esta informação para responder à pergunta. "
                    f"Se nenhuma informação relevante for encontrada, informe o usuário que não foi possível responder. "
                    f"Informação: {results['documents'][0][0]}"
                )
            else:
                return "Nenhuma informação relevante foi encontrada para responder à pergunta."
        
        except ValueError as e:
            return str(e)
        except Exception as e:
            return f"Erro ao processar a pergunta: {str(e)}"


In [5]:
response = (
    "Crie uma resposta final e pergunte se há alguma dúvida sobre o aplicativo SEMPRE JBS"
)


class SmalltalkInput(BaseModel):
    query: Optional[str] = Field(description="consulta do usuário")


class SmalltalkTool(BaseTool):
    name: str = "Smalltalk"
    description: str = "Útil quando o usuário cumprimenta você ou quer bater papo"
    args_schema: Type[BaseModel] = SmalltalkInput

    def _run(
        self,
        query: Optional[str] = None,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Use a ferramenta."""
        return response

    async def _arun(
        self,
        query: Optional[str] = None,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use a ferramenta de forma assíncrona."""
        return response


In [6]:
tools = [EmbeddingTool(), SmalltalkTool()]

Configuração da LLM

In [7]:
llm = ChatOllama(
model='llama3.1',
temperature=0,
)

System Prompts e Question

In [8]:
system_prompt_tool_call = """
Você é um agente de chamada de ferramentas, seu papel é escolher uma ferramenta para resolver a dúvida do usuário.

Você tem acesso às seguintes ferramentas: 
{tools_description_and_args}
Você deve escolher a ferramenta que melhor se adequa à situação.
Não se preocupe, você tem acesso a todas as informações sobre as quais for questionado.
"""


system_prompt_model = """
Contexto: Você é um assistente virtual treinado para ajudar os colaboradores da empresa JBS a esclarecer dúvidas ou resolver problemas sobre os valores da empresa ou o aplicativo de RH chamado Sempre JBS.
Restrito aos dados fornecidos: Responda exclusivamente com base nas informações apresentadas nos exemplos. Não inclua suposições, interpretações ou dados adicionais que não estejam explicitamente descritos nos exemplos fornecidos.
Conciso e direto: Responda de forma objetiva e clara.
Idioma: Sempre em PT-BR.
Tratamento de casos específicos:
- Informações não encontradas ou perguntas fora do contexto: "Não localizei essa informação no documento disponível. Por favor, entre em contato com o RH para mais detalhes."
- Perguntas irrelevantes: "Esta pergunta não se refere aos valores da JBS ou ao aplicativo Sempre JBS. Portanto, não posso respondê-la."
- Perguntas ambíguas: "Por favor, forneça mais detalhes para que eu possa responder com precisão."
Tom da resposta: Formal e educado.

Encontre as informações necessárias para a formulação da resposta nesses exemplos disponíveis: {tool_response}

Responda à pergunta em PT-BR:
"""

question = "Como instalar o Sempre JBS?"

Funções de Execução

In [11]:
def get_tool_call(llm, system_prompt, question, tools):
    system_prompt_formatted = system_prompt.format(tools_description_and_args=render_text_description_and_args(tools).replace('{', '{{').replace('}', '}}'))
    llm_with_tools = llm.bind_tools(tools)
    messages =  [("system", system_prompt_formatted), ("human", question)]
    tool_call = llm_with_tools.invoke(messages)

    return tool_call

def get_tool_response(function_name, function_arguments):    
    if function_name == "Embedding":
        tool_response = EmbeddingTool()._run(question=function_arguments['question'])
    elif function_name == "SmallTalk":
        tool_response = SmalltalkTool()._run(query=function_arguments['question'])
    else:
        tool_response = None

    return tool_response

def get_model_response(llm, tool_response, system_prompt, question):
    system_prompt_formatted = system_prompt.format(tool_response=tool_response)
    messages = [("system", system_prompt_formatted), ("human", question)]
    model_response = llm.invoke(messages)
    
    return model_response

Execução do Agente

In [12]:
tool_call = get_tool_call(llm, system_prompt_tool_call, question, tools)

tool_calls = tool_call.tool_calls
for tool_call in tool_calls:
    function_name = tool_call['name']
    arguments = tool_call['args']

In [13]:
tool_response = get_tool_response(function_name, arguments)

In [14]:
model_response = get_model_response(llm, tool_response, system_prompt_model, question)
model_response.content

'Para instalar o Sempre JBS, você pode seguir os links fornecidos abaixo:\n\n* Para Android: https://play.google.com/store/apps/details?id=br.com.jbs.rh.comvoce&hl=pt_BR&gl=US\n* Para IOS: botm.cc/l/8eQNDoj\n\nEsses links direcionam para as lojas de aplicativos correspondentes, onde você pode baixar e instalar o Sempre JBS.'